# Tag 2 - LLM Fine-Tuning

## Einführung in das Hugging Face Ökosystem

Um die Bestandteile des ML-Workflows und die technischen Details hinter ML-Modellen detaillierter zu beleuchten, haben wir gestern pures PyTorch verwendet, um unsere Daten zu laden und aufzubereiten, und unsere Modelle von Grund auf zu bauen, trainieren, und evaluieren. Für die meisten Aufgabenstellungen ist es heute jedoch üblich, bereits vortrainierte Modelle entweder direkt ohne Anpassungen zu verwenden (auch als *zero-shot* bezeichnet), oder diese für die eigene Problemstellung zu finetunen. 

Zu diesem Zweck hat sich [Hugging Face](https://huggingface.co/) sowohl in der Wissenschaft als auch im kommerziellen Bereich als die primäre Platform herauskristallisiert, um ML-Modelle, Datensätze, etc., auszutauschen. Zudem stellt Hugging Face eine weite Auswahl an Libraries zur Verfügung, die das Verwenden von bestehenden Modellen und Datensätzen, aber auch Aspekte wie Fine-Tuning, Optimierung für die Nutzung in Production, und Training auf Clustern oder spezieller Hardware (z.B. Google TPUs, Habana Gaudi Prozessoren) vereinfacht. 

## Hugging Face Transformers
Die wichtigste Hugging Face Library ist mit Abstand [Transformers](https://huggingface.co/docs/transformers/index). Sie kapselt die Nutzung von vortrainierten Modellen aus verschiedensten Einsatzgebieten, das für sie notwendige Pre- und Post-Processing, sowie ihr Training / Fine-Tuning.  

### Basics
Zur Verwendung eines Modells aus dem Hugging Face Ökosystem benötigt man immer mindestens 2 Komponenten: die richtige Model-Klasse, und die Pre-Processors, die die Inputs für die aktuelle Aufgabenstellung (Bilder, Text, etc.) in das Format umwandeln, dass das Modell erwartet. Laden wir ein Beispiel-Modell, das Language Model RoBERTa (https://huggingface.co/roberta-base):

In [1]:
from transformers import AutoTokenizer, RobertaForMaskedLM

roberta_mlm = RobertaForMaskedLM.from_pretrained("roberta-base")
roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base", add_prefix_space=True)

Die vorangehenden Zeilen laden das RoBERTa Modell, und seinen Pre-Processor, den Tokenizer: Da ein Language Model (grob gesagt) numerischen Input erwartet, muss ein Eingabe-String zuerst in kleinere Teile aufgeteilt werden, die *Tokens*. Je nach Modell kommen hier unterschiedliche Techniken zum Einsatz, bei denen ein Token immer ein gesamtes Wort, oder häufiger den Teil eines Wortes repräsentiert (Sub-Word Tokenization). Die Verwendung der `from_pretrained` Methode mit dem selben Model-Identifier stellt sicher, dass wir die richtige Kombination and Modell und Tokenizer laden.

<span style="color:white; background-color: blue; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Info: </span> Der Tokenizer von RoBERTa ist ein Byte Pair Encoding (BPE) Tokenizer. Diese Technik wurde erstmals von OpenAI GPT-2 verwendet, und wird hier gut erläutert: https://lucytalksdata.com/the-modern-tokenization-stack-for-nlp-byte-pair-encoding/. 

<span style="color:white; background-color: blue; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Info: </span> Um von der Transformer-Architektur, auf der fast alle modernen Language Models basieren, verwendet zu werden, müssen die Tokens (einfache Ids) noch weiter in multi-dimensionale Vektoren umgewandelt werden. Dieser Prozess wird als *Embedding* bezeichnet. Der Prozess von Tokenization zum Embedding wird in diesem Artikel sehr umfangreich erklärt: https://www.lesswrong.com/posts/pHPmMGEMYefk9jLeh/llm-basics-embedding-spaces-transformer-token-vectors-are. 

Wie verwenden wir nun unser Modell? Wie du aus dem Code oben erkennen kannst, haben wir eine Variante von RoBERTa geladen, die für Masked Language Modelling (MLM) finetuned wurde, also das Füllen von Lücken in einem Text. Sehen wir uns ein Beispiel an:

In [2]:
import torch 

inputs = roberta_tokenizer("The capital of France is <mask>.", return_tensors="pt")

with torch.no_grad():
    logits = roberta_mlm(**inputs).logits

mask_token_index = (inputs.input_ids == roberta_tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
roberta_tokenizer.decode(predicted_token_id)

' Paris'

Wie erwartet leiten wir unseren Input-Satz durch den Tokenizer, und geben das Ergebnis des Tokenizers 1-zu-1 and unser Modell weiter. Das Modell gibt für den maskierten Token (in diesem Fall enthält der Satz nur einen) Gewichte zurück (sogenannte Logits), die ausdrücken, welchen Token das Modell an dieser Stelle am wahrscheinlichsten einsetzen würde. Wie erwartet füllt das Modell die Lücke im Text mit der korrekten Antwort: Paris. 

<span style="color:white; background-color: red; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Aufgabe: </span> Sieh dir die `inputs` an das Modell genauer an. Stimmt die Anzahl an Tokens (`input_ids`) mit der Anzahl an Wörtern überein, und wenn nicht, was könnte die Ursache sein (um eine Vermutung zu entkräftigen, sieh dir den Output von `inputs.word_ids(batch_index=0)` an)? Was könnte der Zweck der `attention_mask` sein? 

<span style="color:white; background-color: #FFD700; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Bonus-Aufgabe: </span> Leite dir ab, wie der Code oben genau funktioniert, um an die Antwort des Modells zu kommen. Tipps: Das Modell verarbeitet gewöhnlich nicht nur einen, sondern einen Batch an Sätzen. `tokenizer.mask_token_id` enthält die ID des Tokens, in den `<mask>` umgewandelt wird (also die ID, an der wir interessiert sind). Bei guten Verständnis solltest du den Code so umbauen können, dass du die Antworten für einen Batch an Sätzen auf einmal auslesen kannst. 

<span style="color:white; background-color: red; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Aufgabe: </span> Transformers stellt sogenannte *Pipelines* zur Verfügung, die das Post-Processing der Model-Outputs übernehmen. Nutze die Pipelines API-Doc (https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/pipelines) um das gleiche Ergebnis zu erhalten. Übergib der Pipeline dabei die bereits initialisierten Modell & Tokenizer.

In [3]:
from transformers import pipeline

# Your code goes here

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Weitere Details zum Laden von Modellen

Wie bereits aus dem `with torch.no_grad()` Befehl erkennbar sein sollte, ist das mittels Transformers geladene Modell ein reguläres PyTorch Modell. Wir können uns daher die Details des Modells wie gewohnt ansehen, und auch sonst ohne die Helfer von Transformers mit dem Modell so interagieren, wie wir es von einem PyTorch Modell gewohnt sind:

In [4]:
from torchinfo import summary

summary(roberta_mlm)

Layer (type:depth-idx)                                            Param #
RobertaForMaskedLM                                                --
├─ModuleDict: 1-1                                                 --
├─RobertaModel: 1-2                                               --
│    └─ModuleDict: 2-1                                            --
│    └─ModuleDict: 2-2                                            --
│    └─RobertaEmbeddings: 2-3                                     --
│    │    └─Embedding: 3-1                                        38,603,520
│    │    └─Embedding: 3-2                                        394,752
│    │    └─Embedding: 3-3                                        768
│    │    └─LayerNorm: 3-4                                        1,536
│    │    └─Dropout: 3-5                                          --
│    └─RobertaEncoder: 2-4                                        --
│    │    └─ModuleList: 3-6                                       85,054,464
│   

Diese Model-Summary deutet noch auf ein weiteres Feature von Transformers hin: Viele Modelle wurden nicht nur für einen, sondern mehrere Aufgaben vorbereitet. In unserem Fall sehen wir, dass unser `RobertaForMaskedLM` Modell 2 Bestandteile hat: Das Kern-RoBERTa Modell (`RobertaModel`) und eine Gruppe an zusätzlichen Layern, die den rohen Output des `RobertaEncoder`s in die Logit-Gewichte umwandelt, die wir oben gesehen haben (`RobertaLMHead`). Im Falle von RoBERTa wurden auch *Heads* für andere Aufgabenstellungen spezifiziert, unter anderem für jenen Task, mit dem wir uns als nächstes beschäftigen werden: Named Entity Recognition (NER). 

Um das Modell mit einem Head für NER zu laden, könnten wir die `RobertaForTokenClassification`-Klasse verwenden. Da Transformers die Interfaces der Modelle für die gleiche Aufgabenstellung standardisiert, können wir stattdessen eine der sogenannten `Auto`-Klassen verwenden, die die Details zum Modell automatisch aus seiner Spezifikation lädt:

In [5]:
from transformers import AutoModelForTokenClassification

roberta_ner = AutoModelForTokenClassification.from_pretrained("roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

Wie wir an der Warnung erkennen, wurde zwar das Format für einen NER Head für RoBERTa spezifiziert, der Checkpoint `roberta-base` enthält jedoch keine Weights für diesen Head, weshalb wir ihn erst für unseren spezifischen Einsatzzweck trainieren müssen.

## Named Entity Recognition (NER)

Nach unserem etwas akademischen Beispiel widmen wir uns jetzt einem praktischeren Language-Model Task: Named Entity Recognition, oft auch Token Classification genannt. Hier sollen in einem unstrukturierten Text bestimmte Entitäten (Firmen, Personen, Orte, etc.) erkannt und extrahiert werden. Beispiele für Nutzungs-Szenarien sind:

- Automatische Zuordnung von Support-Anfragen basierend auf den erkannten Regionen, Produkten, Herstellern, etc.
- Automatisches Anreichern mit Kontext-Informationen, z.B. die aktuellen Aktienkurse für erkannte Organisationen in Zeitungs-Artikeln.
- Intelligente Suchfunktionen, die beispielsweise eine medizinische Diagnose-Datenbank nach einer bestimmten Krankheits-Kategorie filtern.


### Hugging Face Datasets

Als Datensatz für unsere folgenden Experimente verwenden wir `bionlp2004` (https://huggingface.co/datasets/tner/bionlp2004), eine Sammlung aus medizinischen Fachtexten, in denen Fachbegriffe für Proteine, DNA, RNA, und Zellen hervorgehoben sind. Dieser Datensatz könnte somit als Beispiel für den Nutzen von NER zur Verwaltung von großen Wissens-Datenbanken gesehen werden, die fachspezifischen Freitext enthalten (man denke an die Confluence-Instanz eines großen Unternehmens). Ein anderer konkreter Use-Case eines Unternehmens wird mit hoher Wahrscheinlichkeit auch einen darauf angepassten Datensatz benötigen, für die Erklärung der technischen Prinzipien ist `bionlp2004` jedoch ausreichend. 

Ähnlich wie Hugging Face Transformers für Modelle gibt Hugging Face Datasets dem Entwickler die Möglichkeit, bekannte Referenz-Datensätze über ein simpel gestaltetes Interface zu laden:

In [51]:
from datasets import load_dataset

bionlp2004 = load_dataset("tner/bionlp2004")

Found cached dataset bionlp2004 (/Users/jan/.cache/huggingface/datasets/tner___bionlp2004/bionlp2004/1.0.0/9f41d3f0270b773c2762dee333ae36c29331e2216114a57081f77639fdb5e904)


  0%|          | 0/3 [00:00<?, ?it/s]

In [52]:
bionlp2004

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 16619
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1927
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 3856
    })
})

Sehen wir uns den geladenen Datensatz etwas genauer an. Der Datensatz wird bereits beim Laden in Train, Validation, und Test Splits aufgeteilt. Jeder dieser Splits ist eine Instanz der `Dataset` Klasse, die effizientes Pre-Processing der enthaltenen Daten ermöglicht und ausgezeichnet mit den restlichen Hugging Face Libraries (wie z.B. Transformers) zusammen arbeitet. 

<span style="color:white; background-color: blue; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Info: </span> Falls man die Features der Datasets Library auch mit eigenen Daten verwenden möchte, gibt es eine Reihe an Möglichkeiten, um ein Dataset zu konstruieren: z.B. aus [diversen Datei-Formaten](https://huggingface.co/docs/datasets/v2.13.1/en/package_reference/loading_methods#from-files), aus [Pandas `DataFrame`s](https://huggingface.co/docs/datasets/v2.13.1/en/package_reference/main_classes#datasets.Dataset.from_pandas), und aus [PyTorch `Dataset`s](https://github.com/huggingface/datasets/issues/4983).



In [53]:
print(bionlp2004["train"][3])

{'tokens': ['Inhibition', 'by', 'anti-VCAM-1', 'monoclonal', 'antibody', '1G11', 'indicated', 'that', 'U937', 'adhesion', 'was', 'VCAM-1', 'dependent', 'and', 'suppression', 'by', 'antioxidants', 'was', 'due', 'to', 'reduced', 'VCAM-1', 'induction', '.'], 'tags': [0, 0, 3, 4, 4, 3, 0, 0, 7, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0]}


Die einzelnen Einträge des Datensatzes enthalten eine Reihe an `"tokens"` (wobei hier `"tokens"` einfach die einzelnen Worte des jeweiligen Texts beschreibt, nicht zu verwechseln mit den numerischen Tokens, die vom Tokenizer unseres Modells produziert werden), und dazugehörige `"tags"`, die jedem `token` eine Entitäts-Klasse zuordnen (oder 0, falls keine zutrifft). Um die numerischen Tag-Werte ihren Labeln zuordnen zu können, bauen wir uns Maps in beide Richtungen.

<span style="color:white; background-color: blue; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Info: </span> Falls im eigenen Datensatz die Input-Texte noch nicht in Wörter unterteilt sind, kann dies beispielsweise mit dem `TreebankWordTokenzier` aus der NLTK Library erreicht werden: https://www.nltk.org/api/nltk.tokenize.TreebankWordTokenizer.html. 

<span style="color:white; background-color: red; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Aufgabe: </span> Unser Modell, RoBERTa, verwendet wie bereits erwähnt Sub-Word Tokenization. Welche Probleme siehst du darin, ein solches Modell für einen Einsatzzweck zu nutzen, in dem immer ganze Wörter oder Wortgruppen kategorisiert werden müssen? 

In [54]:
label2id = {
    "O": 0,
    "B-DNA": 1,
    "I-DNA": 2,
    "B-protein": 3,
    "I-protein": 4,
    "B-cell_type": 5,
    "I-cell_type": 6,
    "B-cell_line": 7,
    "I-cell_line": 8,
    "B-RNA": 9,
    "I-RNA": 10,
}

id2label = {i: label for label, i in label2id.items()}


Damit wir uns die Aufgabenstellung besser vorstellen können, nutzen wir die Library `spacy` um die Entitäten in einem Beispielsatz hervorzuheben.

<span style="color:white; background-color: #FFD700; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Bonus-Aufgabe: </span> Die Token-Klassen folgen dem [IOB2-Schema](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)), in dem bei zusammengehörigen Wörtern das erste mit "B-" Prefix, und alle zugehörigen Folgewörter mit "I-" Prefix klassifiziert werden. Ändere den folgenden Code so ab, dass zusammengehörige Wörter als einzelne Entität angezeigt werden.

In [55]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
from spacy import displacy

ner_colors = {
    "B-DNA": "#FF0000",
    "I-DNA": "#FF0000",
    "B-protein": "#FFA500",
    "I-protein": "#FFA500",
    "B-cell_type": "#088F8F",
    "I-cell_type": "#088F8F",
    "B-cell_line": "#CCCCCC",
    "I-cell_line": "#CCCCCC",
    "B-RNA": "#30D5C8",
    "I-RNA": "#30D5C8"
}

def show_ground_truth_with_spacy(sample):
    detokenizer = TreebankWordDetokenizer()
    text = detokenizer.detokenize(sample["tokens"])
    entities = []
    current_start = 0
    
    for word, tag in zip(sample["tokens"], sample["tags"]):
        if tag != 0:
            word_start = text.find(word, current_start)
            word_end = word_start + len(word)
            entities.append({"label": id2label[tag], "start": word_start, "end": word_end})
            current_start = word_end

    displacy.render({"ents": entities, "text": text}, style="ent", manual=True, options={"colors": ner_colors})

show_ground_truth_with_spacy(bionlp2004["train"][3])

Nachdem wir jetzt wissen, wie unser Ziel-Zustand aussehen soll, wird es Zeit, RoBERTa zu trainieren, um dieses Ziel zu erreichen! 

## LLM - Fine-Tuning

### Transformers Trainer

Gestern haben wir zum Training unseres Modells den kompletten Training-Loop, inklusive dem Laden der Trainings-Daten in Batches, dem Berechnen der Loss-Function und der Backpropagation zum Verbessern der Model-Weights, dem Evaluieren der Model-Weights am Validierungs-Split nach jeder Epoche, und dem Zwischenspeichern von Snapshots, "per Hand" mit PyTorch implementiert. Da sich nahezu alle dieser Aspekte des Trainings von Modell zu Modell kaum unterscheiden, gibt es als Teil der Hugging Face Transformers Library eine `Trainer` Klasse, die uns den Großteil dieser Arbeit abnimmt. Auch werden die Modelle aus dem Hugging Face Hub, die wir mit Transformers laden, mit der richtigen Loss Function ausgeliefert, wodurch wir diese nicht selbst definieren müssen. Ein paar Vorbereitungen müssen wir jedoch noch treffen, bevor wir `Trainer` verwenden können.

### Aufteilung der Wörter auf Tokens
Wie wir bereits gesehen haben, sind unsere Trainings-Daten in Wörter unterteilt. Da unser Model RoBERTa aber mit Sub-Word Tokens arbeitet, müssen wir zuerst unsere Input-Wörter durch RoBERTas Tokenizer schicken, und dann die Tags anpassen, sodass der erste Token eines jeden Wortes den Tag des Wortes zugewiesen bekommt, und alle weiteren Tokens den speziellen Tag `-100` zugeteilt bekommen. Letzteres sorgt dafür, dass diese Tokens in der Loss Funktion ignoriert werden, was hier erwünscht ist, weil sonst längere Wörter einen überproportionalen Einfluss auf den Prediction Loss hätten. Ebenfalls so ignoriert werden Model-spezifische Tokens, die automatisch vom Tokenizer eingefügt werden (wie Start `<s>` und End `</s>` Tokens bei RoBERTa). 

<span style="color:white; background-color: blue; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Info: </span> Der spezielle Wert `-100` is spezifisch für die Loss-Funktion des jeweiligen Modells, und z.B. bei RoBERTa [hier](https://github.com/huggingface/transformers/blob/main/src/transformers/models/roberta/modeling_roberta.py#L926) dokumentiert.

<span style="color:white; background-color: red; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Aufgabe: </span> Versuche, die unten definierte Methode, die einen Batch an conll2003-Einträgen entgegen nimmt, diesen durch RoBERTas Tokenizer schickt, und dann den tokenized Inputs wie oben beschrieben die Labels der ihnen zugehörigen Wörter zuweist. Falls du diesen Schritt überspringen oder deine Lösung kontrollieren willst, führe stattdessen die eingeklappte Notebook-Zelle darunter aus.

In [59]:
def tokenize_and_align_labels(batch):
    # erhalte die in Tokens unterteilten Einträge des Batches
    tokenized_inputs = roberta_tokenizer(batch["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    # dieser Loop durchwandert je einen in tokenized Eintrag pro Iteration
    for i, word_tag_ids in enumerate(batch[f"tags"]):
        # word_ids enthält für jeden Token den Listen-Index des zugehörigen Wortes im Original-Eintrag, oder None, wenn der Token zu keinem Wort gehört (zB. Start und End Tokens)
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        token_tag_ids = []

        # befülle token_tag_ids mit einem Eintrag pro Token: 
        # - die Tag ID des dazugehörigen Wortes (du findest diese in word_tag_ids) für den ersten Token jedes Wortes
        # - -100 für weitere Tokens eines Wortes
        # - -100 für Tokens die keinem Wort zugehören
        
        labels.append(token_tag_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [60]:
def tokenize_and_align_labels(batch):
    tokenized_inputs = roberta_tokenizer(batch["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, word_tag_ids in enumerate(batch[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        token_tag_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                token_tag_ids.append(-100)
            elif word_idx != previous_word_idx:
                token_tag_ids.append(word_tag_ids[word_idx])
            else:
                token_tag_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(token_tag_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Diese Transformation müssen wir nun auf den gesamten Datensatz anwenden. Hierzu hat die `Dataset` Klasse eine `map` Methode, mit der Transformationen effizient in Batches durchführbar sind.

In [62]:
tokenized_bionlp2004 = bionlp2004.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /Users/jan/.cache/huggingface/datasets/tner___bionlp2004/bionlp2004/1.0.0/9f41d3f0270b773c2762dee333ae36c29331e2216114a57081f77639fdb5e904/cache-88b61506ffe3a842.arrow


Loading cached processed dataset at /Users/jan/.cache/huggingface/datasets/tner___bionlp2004/bionlp2004/1.0.0/9f41d3f0270b773c2762dee333ae36c29331e2216114a57081f77639fdb5e904/cache-67b19b7fa535519e.arrow
Loading cached processed dataset at /Users/jan/.cache/huggingface/datasets/tner___bionlp2004/bionlp2004/1.0.0/9f41d3f0270b773c2762dee333ae36c29331e2216114a57081f77639fdb5e904/cache-583acfb8bad7e01f.arrow


### Funktion zur Berechnung von Metriken
Wie gehabt würden wir gerne nach jeder Epoche einen Überblick darüber bekommen, wie unser Modell performt. `Trainer` kann hierzu eine Funktion übergeben werden, die die Ground-Truth Labels und die Predictions des Modells erhält, und daraus beliebige Metriken ableitet. Wir lassen uns die gewohnten Metriken Accuracy, Precision, Recall, und F1 Score wiedergeben, ignorieren aber auch hier jene Tokens, deren Label auf `-100` gesetzt wurde (auch hier um Wörter, die in mehrere Tokens aufgeteilt werden, nicht doppelt zu berücksichtigen).  

In [ ]:
!pip install -U evaluate seqeval accelerate --quiet

In [63]:
import numpy as np
import evaluate

seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    # Uns werden als Predictions für jeden Token die Gewichte (Logits) für alle möglichen Tag IDs zurück gegeben, daher behandeln wir als eigentliche Prediction des Modells für jeden Token jene Tag ID mit dem höchsten Gewicht
    predictions = np.argmax(predictions, axis=2)

    filtered_predictions = []
    filtered_labels = []

    for sample_prediction, sample_label in zip(predictions, labels):
        filtered_predictions.append([
            id2label[p] for p, l in zip(sample_prediction, sample_label) if l != -100
        ])
        filtered_labels.append([
            id2label[l] for p, l in zip(sample_prediction, sample_label) if l != -100
        ])

    results = seqeval.compute(predictions=filtered_predictions, references=filtered_labels)
    
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

### Training
Wir haben die Daten, wir haben die Metriken, unser Modell hat seine Loss-Function bereits eingebaut, somit sind wir startklar! Um `Trainer` zu verwenden, konfigurieren wir es mit einer Instanz von `TrainingArguments`. Diese Config-Klasse enthält diverse bereits bekannte Trainings-Einstellungen wie die Learning-Rate, die Anzahl an Epochs, die trainiert werden soll, und den Pfad, unter dem Snapshots des Modells abgespeichert werden sollen. Nachdem wir ein bereits trainiertes Modell fine-tunen wollen, lassen wir `Trainer` mit einer niedrigen Learning Rate für nur wenige Epochen laufen. 

Das Training selbst kann dann einfach mit Aufruf der `train()` Methode gestartet werden. 

In [64]:
from transformers import AutoModelForTokenClassification 

del roberta_ner
roberta_ner = AutoModelForTokenClassification.from_pretrained(
    "roberta-base", num_labels=len(id2label), id2label=id2label, label2id=label2id
).to("mps")

loading configuration file config.json from cache at /Users/jan/.cache/huggingface/hub/models--roberta-base/snapshots/bc2764f8af2e92b6eb5679868df33e224075ca68/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-DNA",
    "2": "I-DNA",
    "3": "B-protein",
    "4": "I-protein",
    "5": "B-cell_type",
    "6": "I-cell_type",
    "7": "B-cell_line",
    "8": "I-cell_line",
    "9": "B-RNA",
    "10": "I-RNA"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DNA": 1,
    "B-RNA": 9,
    "B-cell_line": 7,
    "B-cell_type": 5,
    "B-protein": 3,
    "I-DNA": 2,
    "I-RNA": 10,
    "I-cell_line": 8,
    "I-cell_type": 6,
    "I-protein": 4,
    "O": 0
  

In [65]:
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification

lr = 2e-5
batch_size = 16
num_epochs = 2

data_collator = DataCollatorForTokenClassification(tokenizer=roberta_tokenizer)

full_training_args = TrainingArguments(
    output_dir="../snapshots/roberta-base-token-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_mps_device=True
)

full_trainer = Trainer(
    model=roberta_ner,
    args=full_training_args,
    train_dataset=tokenized_bionlp2004["train"],
    eval_dataset=tokenized_bionlp2004["validation"],
    tokenizer=roberta_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

full_trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
/Users/jan/Git/ml-workshop/.venv/lib/python3.11/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16619
  Num Epochs = 2
  Instantaneous batch size per d

  0%|          | 0/2078 [00:00<?, ?it/s]

{'loss': 0.1135, 'learning_rate': 1.5187680461982676e-05, 'epoch': 0.48}
{'loss': 0.0711, 'learning_rate': 1.0375360923965351e-05, 'epoch': 0.96}


The following columns in the evaluation set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1927
  Batch size = 16


  0%|          | 0/121 [00:00<?, ?it/s]

...

Saving model checkpoint to ../snapshots/roberta-base-token-classification/checkpoint-2078
Configuration saved in ../snapshots/roberta-base-token-classification/checkpoint-2078/config.json


{'eval_loss': 0.05841284990310669, 'eval_precision': 0.7749451291575216, 'eval_recall': 0.826580226904376, 'eval_f1': 0.7999302892994076, 'eval_accuracy': 0.9489020934991342, 'eval_runtime': 11.4628, 'eval_samples_per_second': 168.109, 'eval_steps_per_second': 10.556, 'epoch': 2.0}


Model weights saved in ../snapshots/roberta-base-token-classification/checkpoint-2078/pytorch_model.bin
tokenizer config file saved in ../snapshots/roberta-base-token-classification/checkpoint-2078/tokenizer_config.json
Special tokens file saved in ../snapshots/roberta-base-token-classification/checkpoint-2078/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ../snapshots/roberta-base-token-classification/checkpoint-2078 (score: 0.05841284990310669).


{'train_runtime': 945.2381, 'train_samples_per_second': 35.164, 'train_steps_per_second': 2.198, 'train_loss': 0.07329441837901904, 'epoch': 2.0}


TrainOutput(global_step=2078, training_loss=0.07329441837901904, metrics={'train_runtime': 945.2381, 'train_samples_per_second': 35.164, 'train_steps_per_second': 2.198, 'train_loss': 0.07329441837901904, 'epoch': 2.0})

Mission Accomplished! ... Oder? 

Unser "naives" Fine-Tuning lässt sich mit RoBERTa zwar umsetzen, skaliert aber nicht für Fine-Tuning Projekte mit größeren Modellen:
- Mit unserem Ansatz adjustieren wir alle Parameter des Models. `roberta-base` hat ~163 Millionen Parameter, und ist damit nach heutigen Standards ein sehr kleines Modell: Modelle wie LLaMa, FLAN-UL2, Falcon, etc., haben oft 7 Milliarden Parameter in ihrer kleinsten Variante, und größere Varianten mit über 50 Milliarden Parametern. 7 Milliarden Parameter in Float-16 verbrauchen bereits 11-12 GB an Speicher für Inferenz (also ohne jegliche Gradient-Berechnung), somit wäre das Fine-Tuning solcher Modelle auf den meisten handelüblichen Grafikkarten unmöglich, da die zusätzlichen Daten für die Gradient-Berechnung beim Training die 16-24 GB GPU-Memory sprengen würde. 
- Da wir alle Parameter unseres Modells tunen, müssen wir nun auch alle diese Parameter für jeden unserer Nutzungszwecke ausliefern. Nutzen wir also zB die größte Variante von LLaMa (65B), müssten wir alle 65 Milliarden Parameter für jeden unserer Finetuning-Projekte speichern (über 100 GB).

Inzwischen gibt es eine Reihe an Techniken, die unter den Sammelbegriff *Parameter-Efficient Fine-Tuning* fallen, um das Tuning aller Modell-Parameter bei Large Language Models zu vermeiden.

## Parameter-Efficient Fine-Tuning (PEFT)

PEFT Methoden basieren alle auf dem Prinzip, einem bereits trainierten Modell an unterschiedlichen Stellen neue Parameter anzufügen, und nur diese beim Fine-Tuning zu trainieren. In diese Kategorie fallen:
- Soft Prompt Tuning
- Prefix Tuning
- Adapter
- LLaMa Adapter (eine Kombination aus den Prinzipien von Prefix Tuning und Adaptern)
- Low-Rank Adaptation (LoRA)

Durch ihr Funktionsprinzip mindern PEFT Methoden die oben angesprochenen Probleme mit Fine-Tuning von LLMs: Da nur ein Bruchteil der Model-Parameter optimiert wird, sinkt besonders bei großen Modellen der Speicherbedarf für die Gradient-Berechnung enorm, was das Training auf deutlich schwacherer Hardware möglich macht. Und um unterschiedliche finetuned Varianten des Modells zu erhalten, ist es nur mehr nötig, die Werte der zusätzlichen PEFT Parameter auszutauschen. 

Im folgenden verwenden wir LoRA, um unser RoBERTa Modell zu tunen. Die technischen Details hinter den einzelnen PEFT-Methoden sind teils etwas kompliziert, glücklicherweise gibt es aber auch hier bereits Libraries, die die Anwendung der Methoden stark vereinfachen. Populär sind unter anderem [Adapter-Transformers](https://github.com/adapter-hub/adapter-transformers) und [Hugging Face PEFT](https://github.com/huggingface/peft). Wir verwenden zweitere, da sich mit der PEFT Library die nötigen Änderungen zu unserem bisherigen Setup darauf beschränken, unser Model mit einem `get_peft_model()` Aufruf zu transformieren:

In [56]:
from peft import get_peft_model, LoraConfig, TaskType

peft_config_lora = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1, bias="all"
)

roberta_ner_lora = get_peft_model(roberta_ner, peft_config_lora).to("mps")
roberta_ner_lora.print_trainable_parameters()

trainable params: 1080342 || all params: 355119126 || trainable%: 0.30421960432511314


Wie der Output oben zeigt, trainieren wir nun weniger als 0.5% der Parameter des Modells, wodurch die Gradient-Berechnung von PyTorch deutlich vereinfacht wird. Dies äußert sich bei großen Modellen primär im verringerten Speicherverbrauch, kann aber auch zu kürzeren Trainings-Zeiten führen (bei identer Anzahl an Epochen; die AutorInnen des LoRA-Papers beispielsweise bemerkten einen 25% Speedup beim Fine-Tuning von GPT-3 175B). 

Der restliche Trainings-Code bleibt gleich, jedoch erhöhen wir die Learning-Rate und die Anzahl an Epochen, da wir hier nicht Gefahr laufen, die bereits trainierten Modell-Parameter übermäßig zu verändern. 

<span style="color:white; background-color: red; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Aufgabe: </span> Beobachte das Training mittels LoRA im Vergleich zu unserem vorherigen Durchlauf. Läuft das Training schneller/langsamer? Wie sieht der GPU-Speicherverbrauch aus? Ist die Accuracy am Validation-Set merkbar niedriger für das LoRA-Model? 

<span style="color:white; background-color: red; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Aufgabe: </span> Inspiziere die Files der beim Training erstellten Snapshots. Wie unterscheiden sie sich von den Snapshots des Fine-Tunings ohne PEFT-Methoden? 

In [13]:
from transformers import TrainingArguments, Trainer

lr = 1e-3
batch_size = 16
num_epochs = 5

training_args_lora = TrainingArguments(
    output_dir="roberta-base-lora-token-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_mps_device=True
)

trainer_lora = Trainer(
    model=roberta_ner_lora,
    args=training_args_lora,
    train_dataset=tokenized_bionlp2004["train"],
    eval_dataset=tokenized_bionlp2004["validation"],
    tokenizer=roberta_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer_lora.train()

## Inferenz 
Unsere finetuned Modelle können wir nun am Test-Datensatz ausprobieren. Hier laufen wir jedoch in die gleiche Problematik, die wir auch schon im Pre-Processing hatten: Unser Model liefert uns Klassifizierungen auf Token-Ebene, diese müssen wir zum Anzeigen jedoch zuerst wieder auf Wort-Level zusammenfassen. Hierfür gibt es jedoch eine Pipeline, deren Output wir einfach in das Format der `spacy`-Library umwandeln können, um unsere NER-Ergebnisse darzustellen.

<span style="color:white; background-color: #FFD700; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Bonus-Aufgabe: </span> Setze die Umwandlung der Token-Level Outputs des Models in Klassifizierungen auf Wort-Ebene selbst um. 

In [75]:
from transformers import pipeline

token_classifier = pipeline("token-classification", model=roberta_ner, tokenizer=roberta_tokenizer, aggregation_strategy="simple", device="mps")

In [80]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

test_text = TreebankWordDetokenizer().detokenize(bionlp2004["test"][0]["tokens"])
token_classifier(test_text)

[{'entity_group': 'protein',
  'score': 0.9465647,
  'word': ' glucocorticoid receptors',
  'start': 10,
  'end': 34},
 {'entity_group': 'cell_type',
  'score': 0.9518865,
  'word': ' lymphocytes',
  'start': 38,
  'end': 49}]

In [79]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
from spacy import displacy

ner_colors_combined = {
    "DNA": "#FF0000",
    "protein": "#FFA500",
    "cell_type": "#088F8F",
    "cell_line": "#CCCCCC",
    "RNA": "#30D5C8",
}

def show_result_with_spacy(sample, pipeline):
    detokenizer = TreebankWordDetokenizer()
    
    text = detokenizer.detokenize(sample["tokens"])
    result = pipeline(text)
    entities = [{"start": e["start"], "end": e["end"] , "label": e["entity_group"]} for e in result]
    
    displacy.render({"ents": entities, "text": text}, style="ent", manual=True, options={"colors": ner_colors_combined})

show_result_with_spacy(bionlp2004["test"][0], token_classifier)

## Weitere Ressourcen

#### Dokumentation
- Hugging Face Transformers: https://huggingface.co/docs/transformers/index
- Hugging Face Datasets: https://huggingface.co/docs/datasets/index
- Hugging Face PEFT: https://huggingface.co/docs/peft/index
- Adapter-Transformers (eine Library für die Adapter-Methode für PEFT): https://docs.adapterhub.ml/index.html

#### Theorie
- Parameter-Efficient Fine-Tuning (PEFT):
  - Eine Serie an ausgezeichneten Artikeln über die Intuition hinter den diversen Methoden:
    - https://magazine.sebastianraschka.com/p/finetuning-large-language-models
    - https://magazine.sebastianraschka.com/p/understanding-parameter-efficient
    - https://magazine.sebastianraschka.com/p/finetuning-llms-with-adapters
    - https://lightning.ai/pages/community/article/understanding-llama-adapters/
  - Die ursprünglichen Papers der diversen Methoden:
    - Soft Prompt-Tuning: https://arxiv.org/abs/2104.08691
    - Prefix Tuning: https://arxiv.org/abs/2101.00190
    - Adapters: https://arxiv.org/abs/1902.00751
    - LoRa: https://arxiv.org/abs/2106.09685
- Transformer:
  - Eine weniger technische, dafür intuitive Erklärung der Transformer-Architektur: https://jalammar.github.io/illustrated-transformer/
  - Eine sehr technische Zusammenfassung der Transformer Architektur: https://lilianweng.github.io/posts/2023-01-27-the-transformer-family-v2/
  - Das originale Transformer Paper mit Code-Umsetzung: http://nlp.seas.harvard.edu/annotated-transformer/
